In [1]:
# scraping 
import requests
from bs4 import BeautifulSoup

# parsing
import re
import dateutil.parser
from word2number import w2n
import string

# storing
import pickle

# analysis -- for other file, just here for testing
import pandas as pd

In [2]:
url = 'http://data.newsday.com/long-island/data/entertainment/movies/oscar-winners-history/?currentRecord=1'

In [3]:
def get_movie_page(url):
    '''
    Gets movie page as soup object for the provided url. 
    
    Args:
        url (str): url of movie page
    Returns: 
        movie_page (soup object): html of movie page
    '''
    movie_page = BeautifulSoup(requests.get(url).text, 'lxml')
    return movie_page

In [63]:
pgs = xrange(1, 2563, 50)

In [73]:
def get_stuff(pgs):
    alldata = []
    for pg in pgs:
        url = 'http://data.newsday.com/long-island/data/entertainment/movies/' + \
            'oscar-winners-history/?currentRecord=' + str(pg)
        soup = get_movie_page(url)
        rows = soup.select('.sdb-even,.sdb-odd')
        alldata.extend([get_data(row) for row in rows])
    return alldata

In [76]:
def get_data(row):
    return {
        'year': row.select('.sdb-field-year')[0].text,
        'category': row.select('.sdb-field-category')[0].text,
        'won': row.select('.sdb-field-result')[0].text.lower() == 'yes',
        'film': row.select('.sdb-field-filmName')[0].text,
        'person': row.select('.sdb-field-actorDirectorName')[0].text
    }


In [82]:
with open('pickled_data/oscars-data.pkl', 'r') as picklefile:
     data = pickle.load(picklefile)

In [83]:
data = pd.DataFrame(data)

In [166]:
data.head(10)

,category,film,person,won,year
0,Best Picture,"""Captain Phillips""",,False,2013
1,Best Picture,"""Gravity""",,False,2013
2,Best Picture,"""Nebraska""",,False,2013
3,Best Picture,"""12 Years a Slave""",,True,2013
4,Best Actor,"""American Hustle""",Christian Bale,False,2013
5,Best Actor,"""The Wolf of Wall Street""",Leonardo DiCaprio,False,2013
6,Best Actor,"""Dallas Buyers Club""",Matthew McConaughey,True,2013
7,Best Actress,"""Blue Jasmine""",Cate Blanchett,True,2013
8,Best Actress,"""Philomena""",Judi Dench,False,2013
9,Best Supporting Actor,"""Captain Phillips""",Barkhad Abdi,False,2013


In [195]:
# get actors
actors = data[data.category.str.contains('Act')]
actors.year = actors.year.apply(int)

In [141]:
# get directors 
dirs = data[data.category == 'Best Director']
dirs.year = dirs.year.apply(int)

'Hi'

In [177]:
def count_nominations(df, person, year):
    """Total nominations before a given year for an actor or director.
    args:
        df (dataframe): data frame
        person (string): director or actor name name
        year (int): cutoff year
    returns:
        (int) number of nominations
    """
    return df[df.person == person][df.year < year].shape[0]

In [ ]:
df = pd.DataFrame({})

In [193]:
actors.loc[14, 'year']

2013

In [207]:
test = actors.head()

test['oscars_before'] = test.apply(lambda row: count_nominations(actors, row.person, row.year), axis=1)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [208]:
count_nominations(actors, 'Cate Blanchett', 2013)

5

In [204]:
test

,category,film,person,won,year,oscars_before
4,Best Actor,"""American Hustle""",Christian Bale,False,2013,1
5,Best Actor,"""The Wolf of Wall Street""",Leonardo DiCaprio,False,2013,3
6,Best Actor,"""Dallas Buyers Club""",Matthew McConaughey,True,2013,0
7,Best Actress,"""Blue Jasmine""",Cate Blanchett,True,2013,5
8,Best Actress,"""Philomena""",Judi Dench,False,2013,6


In [185]:
actors[actors.person == 'Meryl Streep']

,category,film,person,won,year
34,Best Actress,"""August: Osage County""",Meryl Streep,False,2013
92,Best Actress,"""The Iron Lady""",Meryl Streep,True,2011
158,Best Actress,"""Julie & Julia""",Meryl Streep,False,2009
199,Best Actress,"""Doubt""",Meryl Streep,False,2008
279,Best Actress,"""The Devil Wears Prada""",Meryl Streep,False,2006
361,Best Supporting Actress,"""Adaptation""",Meryl Streep,False,2002
484,Best Actress,"""Music of the Heart""",Meryl Streep,False,1999
499,Best Actress,"""One True Thing""",Meryl Streep,False,1998
594,Best Actress,"""The Bridges of Madison County""",Meryl Streep,False,1995
744,Best Actress,"""Postcards from the Edge""",Meryl Streep,False,1990


In [164]:
dirs_names = dirs['person']
dirs_years = pd.get_dummies(dirs['year'])

In [155]:
poop = pd.concat((dirs_names, dirs_years), axis=1, join='inner')

In [156]:
poop.head()

,person,1928,1929,1930,1931,1932,1933,1934,1935,1936,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
14,David O. Russell,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
15,Alexander Payne,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
16,Martin Scorcese,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
40,Alfonso Cuaron,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
41,Steve McQueen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


,1928,1929,1930,1931,1932,1933,1934,1935,1936,1937,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
person,,,,,,,,,,,,,,,,,,,,,
Adrian Lyne,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Akira Kurosawa,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Alan J. Pakula,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Alan Parker,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
Alejandro Gonz&aacute;lez I&ntilde;&aacute;rritu,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Alexander Hall,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Alexander Payne,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
Alfonso Cuaron,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Alfred Hitchcock,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
